### Imports

In [1]:
import sys

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
from matplotlib.ticker import ScalarFormatter

from astropy import units as u
from astropy import constants as const
from astropy.table import QTable

from pathlib import Path
import os

sys.path.insert(0, '/home/emelie/Desktop/Codes/PETAR/PeTar-master/tools/')
from analysis import *

In [2]:
plt.rcParams.update({'xtick.labelsize':13, 'ytick.labelsize':13, 'axes.titlesize':16, 
                     'axes.grid':True, 'axes.labelsize':15, 'legend.fontsize':13})

### Mahalanobis distance

The distance in 6D phace space is given by the Mahalanobis distance: 

\begin{equation}
    d_{M} (\mathbf{x_1}, \mathbf{x_2}:C) = \sqrt((\mathbf{x_1} - \mathbf{x_2})C^{-1}(\mathbf{x_1} - \mathbf{x_2})^T)
\end{equation}

where $\mathbf{x_1}$ and$\mathbf{x_2}$ are two points in 6D phace space and $C$ is given by a $6 \times 6$ matrix with elements following:

\begin{equation}
    C_{ij} =  \langle(x_i - <x_i>) (x_j - <x_j>) \rangle,
\end{equation}

where $\langle \rangle$ represent the mean value. This gives a matrix like:

\begin{equation}
    C =  \begin{pmatrix}
            C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
            C_{21} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
            C_{31} & C_{32} & C_{33} & C_{34} & C_{35} & C_{36} \\
            C_{41} & C_{42} & C_{43} & C_{44} & C_{45} & C_{46} \\
            C_{51} & C_{52} & C_{53} & C_{54} & C_{55} & C_{56} \\
            C_{61} & C_{62} & C_{63} & C_{64} & C_{65} & C_{66} \\
         \end{pmatrix}.
\end{equation}


I have $d_{nj} C^{-1}_{ji}d^{T}_{in}$ and want to sum such that it has shape (n). 

In [ ]:
def mahalanobis_distance(first_pnt, space_6d):
    """
    Summary: 
    Calculates the Mahalanobis distance between one point (first_point) and other points closeby (space_6d)
    ----------------------------------------------------------
    Parameters:
    -----------
    first_pnt: 1d array
        Array with 6d phase space data (x, y, z, vx, vy, vz) for first point 
    
    space_6d: 2d-array
        Array with 6d phase space data (x, y, z, vx, vy, vz) for closeby points
        
    Returns:
    --------
    mahalanobis_dist: 1d array
        Mahalanobis distances between each closeby point and the first point
        
    """
    
    # Calculate the difference vector 
    diff = first_pnt - space_6d
    
    covariance_matrix = np.cov(space_6d, rowvar=False)
    inv_covariance_matrix = np.linalg.inv(covariance_matrix)
    
    
    # Calculate the Mahalanobis distance
    mahalanobis_dist = np.sqrt(np.einsum('nj,ji,in->n', diff, inv_covariance_matrix, diff.T))

    return mahalanobis_dist

### Testing Mahalanobis

In [ ]:
space_data_6d = run_fid_large_mod_data[:, 1:7, -1]
max_z_pos = np.where(space_data_6d[:, 2]==np.max(space_data_6d[:, 2]))[0]

first_point = space_data_6d[0, :]
space_6d_data2 = space_data_6d
print(np.shape(space_6d_data2))
space_6d_data2 = np.delete(space_6d_data2, max_z_pos, axis=0)
print(np.shape(space_6d_data2))


data1 = space_6d_data2[0:100000, :]
data2 = space_6d_data2[100001:200000, :]
data3 = space_6d_data2[200001:300000, :]
data4 = space_6d_data2[300001:400000, :]
data5 = space_6d_data2[400001:500000, :]
data6 = space_6d_data2[500001:600000, :]
data7 = space_6d_data2[600001:700000, :]
data8 = space_6d_data2[700001:800000, :]
data9 = space_6d_data2[800001:900000, :]
data10 = space_6d_data2[900001:-1, :]


datas = [data1, data2, data3, data4, data5, data6, data7, data8, data9, data10]


beg = np.array([0, 100001, 200001, 300001, 400001, 500001, 600001, 700001, 800001, 900001])
end = np.array([100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, -1])

In [ ]:
dms = np.zeros((len(space_6d_data2)))

for i, data in enumerate(datas):
    dm = mahalanobis_distance(first_point, data)
    dms[beg[i]:end[i]] = dm

    
print(np.shape(dms))
print(dms)

### Plotting Mahalanobis distances from first point to the rest of the dataset

In [ ]:
fig3, ax3 = plt.subplots(figsize=(11, 10))


sc = ax3.scatter(space_6d_data2[:,0,], space_6d_data2[:,2,], c=dms, s=1, 
                 cmap='bwr')
plt.colorbar(sc, label='6D distance')
ax3.set_xlim(xmin=-0.2e5, xmax=0.7e5) #xmin=-0.7e5, xmax=0.8e5
ax3.set_ylim(ymin=-0.85e5)
ax3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax3.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax3.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax3.yaxis.get_major_formatter().set_powerlimits((0, 0))

#ax3.set_title('')
ax3.set_xlabel('x [pc]')
ax3.set_ylabel('z [pc]')

plt.show()

### Stream walk function

In [ ]:
def stream_walk(space_6d, max_city_dist, max_block_dist):
    """
    Summary:
    ---------
    City means a larger amount of particles that are closeby, the particles in the city are the citizens. 
    Defining this makes other calculations much faster as it limits the amount of particles. 
    Citizens are found by x, y, z coordinates and radial distance. 
    The block contains the closest particles, the neighbours. I choose the next particle from this category.
    Neighbours are foudn through the Mahalanobis distance. 
    ...
    ----------------------------------------------------------
    Parameters:
    -----------
    space_6d: 2d-array
        Array with space coordinates with columns x, y, z, vx, vy, vz
        
    max_city_dist: float or int
        Maximum distance to particles that are closeby to save computation time
        
    max_block_dist: float or int
        Maximum distance to particles that are neighbours to the particle
        
    Return:
    --------
    path: 2d array
        Array with points along the path of the stream
    """
    
    space_6d_copy = space_6d.copy() # copying space data to keep input array intact
    #print(np.shape(space_6d_copy))
    
    # FIRST PARTICLE
    #-------------------------------------------------------------------------------------------
    # Finding first particle position, defining it as z_max
    first_particle_pos = np.where(space_6d[:, 2]==np.max(space_6d[:, 2]))[0][0] 
    # Extracting first particle information
    first_particle = space_6d_copy[first_particle_pos, :] 
    # Creating path array with first particle at the beginning
    path = np.array([first_particle]) 
    # Removing first particle from list
    space_6d_copy = np.delete(space_6d_copy, first_particle_pos, axis=0)
    #print(np.shape(space_6d_copy))
    
    
    # SECOND PARTICLE: need two first particles to initiate loop
    #-------------------------------------------------------------------------------------------
    # Extracting first particle data
    previous_particle = path[-1, :]
    
    ## Finding city particles ##
    # 3D distances - difference
    difference = previous_particle[np.newaxis, :] - space_6d_copy
    # 3D distances - r = sqrt(x² + y² + z²)
    distance_3d = np.sqrt(difference[:, 0]**2 + difference[:, 1]**2 + difference[:, 2]**2)
    # Finding the particles in the city
    city_mask = distance_3d<=max_city_dist
    # Extracting the citizen data
    citizens = space_6d_copy[city_mask]
    print(f'c1 = {len(citizens)}')
    # Extracting the positions of the citizens in the whole data set 
    citizens_positions = np.where(distance_3d<=max_city_dist)[0]
    # Mahalanobis distances to citizens
    dist_m = mahalanobis_distance(previous_particle, citizens)
    # Fincing closest citizen
    min_distance = np.min(dist_m)
    # Position of closest citizen
    min_distance_pos = np.where(dist_m==min_distance)[0]
    
    # Extracting second particle data
    second_particle = citizens[min_distance_pos, :]
    # Adding to path
    path = np.vstack([path, second_particle])
    # Remove from particle list
    space_6d_copy = np.delete(space_6d_copy, min_distance_pos, axis=0) 
    
    # SETTING UP LOOP CONDITION
    #--------------------------------------------------------------------------------------------
    # Previous particle, here second
    previous_particle = path[-1, :]
    # 3D distances -> difference
    difference = previous_particle[np.newaxis, :] - space_6d_copy
    # 3D distances -> r = sqrt(x² + y² + z²)
    distance_3d = np.sqrt(difference[:, 0]**2 + difference[:, 1]**2 + difference[:, 2]**2)
    print(np.min(distance_3d), np.max(distance_3d))
    # Finding the particles in the city
    city_mask = distance_3d<=max_city_dist
    # Extracting the citizen data
    citizens = space_6d_copy[city_mask]
    # Extracting the positions of the citizens in the whole data set 
    citizens_positions = np.where(distance_3d<=max_city_dist)[0]
    print(f'c2 = {len(citizens)}')
    
    # Mahalanobis distances to citizens
    dist_m = mahalanobis_distance(previous_particle, citizens)
    print(np.min(dist_m), np.max(dist_m))
    
    
    # While there are particles closer within block distances !!!MIGHT HAVE TO CHANGE!!!
    while any(dist_m<=max_block_dist): 
        print('Starting loop')
        # Finding neighbours
        neighbours_mask = (dist_m<=max_block_dist)
        # Extracting neighbours
        neighbours = citizens[neighbours_mask, :] 
        # Mahalanobis distance from previous particle
        neighbours_dm = dist_m[neighbours_mask]
        # Finding neighbour's positions in city
        neighbours_city_positions = np.where(dist_m<=max_block_dist)[0]
        # Finding neighbour's positions in data
        neighbours_positions = citizens_positions[neighbours_city_positions]
        print(f'n = {len(neighbours)}')
        
        
        # If there are "few" neighbours
        if len(neighbours)<=1500:
            # Finding minimum distance to other neighbours
            min_distance = np.min(neighbours_dm) 
            # Finding where closest neighbour
            min_distance_pos = np.where(neighbours_dm==min_distance)[0] 
            # Data for closest neighbour
            closest_particle = neighbours[min_distance_pos, :]
            #closest_particle_pos = neighbours_positions[min_distance_pos]
            # Adding neighbour to stream path
            path = np.vstack([path, closest_particle])
            
            closest_neighbours_mask = neighbours_dm<=0.5*max_block_dist
            closest_neighbours_pos = neighbours_positions[closest_neighbours_mask]
            # Removing all neighbouring particles from list of particles
            space_6d_copy = np.delete(space_6d_copy, closest_neighbours_pos, axis=0)
        
        # If there are very many neighbours
        else:
            # Number of citizens for each neighbour
            densities = np.zeros(len(neighbours))
            volume = np.zeros((len(neighbours), len(space_6d_copy)))
            
            ### DOES NOT WORK! ###
            # Differences in 3D (x, y, z)
            diffs = neighbours[:, np.newaxis, :3] - space_6d_copy[:, :3]
            #print(np.shape(diffs)) # Shape: (N_n, N_all, 3D)
            # Calculating r
            dists_squared = np.sum(diffs**2, axis=2) # Need shape: (N_n, N_all)
            #print(np.shape(dists_squared))
            dists_av = np.sqrt(dists_squared)
            #print(np.min(dists_av), np.max(dists_av))
            # Finding city limits
            citizens_mask = dists_av <= max_city_dist
            non_citizens_mask = dists_av > max_city_dist
            # Finding closest particles
            volume[citizens_mask] = 1
            volume[non_citizens_mask] = 0
            # Calculating number of citizens for each neighbour
            densities = np.sum(volume, axis=1)
            #print(f'densities = {densities}')
            # Finding neighbour with most citizens
            max_dens = np.where(densities==np.max(densities))[0]
            #print(max_dens)
            
            # If there are more than one neighbour with maximum citizens
            if len(max_dens)>1: 
                # Neighbours with maximum citizens
                max_dens_neighbours = neighbours[max_dens, :]
                # Taking the first close neighbour with maximum citizens
                closest_particle = max_dens_neighbours[0, :]
                # Adding closest neighbour to stream path
                np.vstack([path, closest_particle]) 
            
            # If there is only one neighbour with maximum citizens
            else:
                # Neighbour in right direction
                closest_neighbour = neighbours[max_dens, :]
                # Adding closest neighbour to stream path
                path = np.vstack([path, closest_neighbour]) 
        
        
        
            # Removing all neighbouring particles from list of particles
            space_6d_copy = np.delete(space_6d_copy, neighbours_positions, axis=0)
        
        fig, ax = plt.subplots(figsize=(10, 8))
        
        ax.scatter(space_6d_copy[:, 0], space_6d_copy[:, 2], c='b', alpha=0.5, s=1)
        ax.plot(path[:, 0], path[:, 2], color='r', marker='x')
        
        ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.xaxis.get_major_formatter().set_powerlimits((0, 0))
        ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.yaxis.get_major_formatter().set_powerlimits((0, 0))

        ax.set_title('Stream walk path and stream data')
        ax.set_xlabel('x [pc]')
        ax.set_ylabel('z [pc]')
        plt.show()
        
        
        # Setting new distances for the loop
        previous_particle = path[-1, :]
        # 3D distances -> difference
        difference = previous_particle[np.newaxis, :] - space_6d_copy
        # 3D distances -> r = sqrt(x² + y² + z²)
        distance_3d = np.sqrt(difference[:, 0]**2 + difference[:, 1]**2 + difference[:, 2]**2)
        # Finding the particles in the city
        city_mask = distance_3d<=max_city_dist
        # Extracting the citizen data
        citizens = space_6d_copy[city_mask]
        # Extracting the positions of the citizens in the whole data set 
        citizens_positions = np.where(distance_3d<=max_city_dist)[0]
        print(f'c = {len(citizens)}')
        print()
    
        # Mahalanobis distances to citizens
        dist_m = mahalanobis_distance(previous_particle, citizens)
    
    return path

### Testing stream walk function

In [ ]:
space_data_6d = run_fid_large_mod_data[:, 1:7, -1]


stream_path = stream_walk(space_data_6d, max_city_dist=0.4e5, max_block_dist=5)
print(np.shape(stream_path))

In [ ]:
fig2, ax2 = plt.subplots(figsize=(9, 8))


sc = ax2.scatter(run_fid_large_mod_data[:,1,-1], run_fid_large_mod_data[:,3,-1], c='b', alpha=0.3, s=1)

plt.colorbar(sc, label='6D distance')

ax2.plot(stream_path[:, 0], stream_path[:, 2], color='r', marker='x')
#ax2.scatter(run_fid_large_mod_data[645881,1,-1], run_fid_large_mod_data[645881,3,-1], c='b')
#ax2.set_xlim(xmin=-0.7e5, xmax=0.8e5) #xmin=-0.7e5, xmax=0.8e5
#ax2.set_ylim(ymin=-1.0e5, ymax=1.75e5)
ax2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax2.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax2.yaxis.get_major_formatter().set_powerlimits((0, 0))

#ax2.set_title('')
ax2.set_xlabel('x [pc]')
ax2.set_ylabel('z [pc]')

plt.show()

### Core evolution code vs1

In [ ]:
# Mean positions of galaxy form header
fid_cluster_mean_pos = fid_mod_header[2:5, 0]
print(np.shape(fid_cluster_mean_pos))

# Centering galaxy in coordinate system using header positions
fid_cluster_pos = fid_mod_data[:, 1:4, 0] - fid_cluster_mean_pos

# Calculating distance from the center to all particles in galaxy
fid_part_pos_in_cluster = np.sqrt(np.sum(fid_cluster_pos**2, axis=1))
print(np.shape(fid_part_pos_in_cluster))
print(np.min(fid_part_pos_in_cluster))
print(np.max(fid_part_pos_in_cluster))

# Which particles are inside certain radius of core
core_part_rows = np.where(fid_part_pos_in_cluster<=(2.17e-3*1))[0]

# Extracting core particles
fid_core_part = fid_cluster_pos[core_part_rows, :]
print(np.shape(fid_core_part))

In [ ]:
figt1 = plt.figure(figsize=(10, 8))
axt1 = figt1.add_subplot(projection='3d', computed_zorder=False)

#axt1.scatter(fid_cluster_pos[:, 0], fid_cluster_pos[:, 1], fid_cluster_pos[:, 2], s=5, c='b', alpha=0.05)
sc = axt1.scatter(fid_core_part[:, 0], fid_core_part[:, 1], fid_core_part[:, 2], s=10, c=fid_core_part[:, 1], 
             alpha=0.5, cmap='spring')

plt.colorbar(sc, label='y [kpc]')
axt1.set_xlabel('x [kpc]', labelpad=15)
axt1.set_ylabel('y [kpc]', labelpad=15)
axt1.set_zlabel('z [kpc]', labelpad=5)
axt1.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt1.xaxis.get_major_formatter().set_powerlimits((0, 0))
axt1.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt1.yaxis.get_major_formatter().set_powerlimits((0, 0))
axt1.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt1.zaxis.get_major_formatter().set_powerlimits((0, 0))

axt1.view_init(4, 45)


plt.show()

calculate distance between all core particle and extract the longest distance for every timestep and see how it evolves over time and see how/if the core is being disrupted and when. 

In [ ]:
fid_core_part_all = np.empty((len(core_part_rows), 4, 40))
mean_dists = np.empty((40))

for i in range(40):
    # Shape N, 3, 1
    tstep_pos = fid_mod_data[core_part_rows, 0:4, i]
    fid_core_part_all[:, :, i] = tstep_pos
    
    # calculate distance between all core patricles in x, y, and z
    diffs = tstep_pos[:, np.newaxis, 1:] - tstep_pos[np.newaxis, :, 1:]
    #print(diffs)
    #Calculates the r-distance between all core particles
    #This gives a double-sided square array divided by the diagonal, which is 0
    dists = np.linalg.norm(diffs, axis=2)
    #print(dists)
    #print()
    mean_dists[i] = np.mean(dists[np.triu_indices(10, k=1)])
    
    
    
print(np.shape(fid_core_part_all))
print(np.shape(mean_dists))
print(mean_dists)

In [ ]:
figt3 = plt.figure(figsize=(10, 8))
axt3 = figt3.add_subplot(projection='3d', computed_zorder=False)

#axt1.scatter(fid_cluster_pos[:, 0], fid_cluster_pos[:, 1], fid_cluster_pos[:, 2], s=5, c='b', alpha=0.05)
sc = axt3.scatter(fid_core_part_all[:, 1, -1], fid_core_part_all[:, 2, -1], fid_core_part_all[:, 3, -1], 
                  s=10, c=fid_core_part_all[:, 2, -1], alpha=0.5, cmap='spring')

plt.colorbar(sc, label='y [kpc]')
axt3.set_xlabel('x [kpc]', labelpad=20)
axt3.set_ylabel('y [kpc]', labelpad=20)
axt3.set_zlabel('z [kpc]', labelpad=10)
axt3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt3.xaxis.get_major_formatter().set_powerlimits((0, 0))
axt3.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt3.yaxis.get_major_formatter().set_powerlimits((0, 0))
axt3.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt3.zaxis.get_major_formatter().set_powerlimits((0, 0))

axt3.view_init(4, 45)


plt.show()